In [3]:
import pandas as pd 
import chardet
import matplotlib.pyplot as plt 
import cld3
import re 
import unicodedata
import string
import cld3
import gensim
import torch 
from transformers import BertModel
from transformers import BertTokenizerFast
import numpy as np

/home/suyash/anaconda3/envs/Pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
df=pd.read_csv('/home/suyash/Desktop/Tweets/Preprocessing/sample.csv')

In [32]:
df.head(10)

,Unnamed: 0,index,target,ids,date,flag,user,text
0,0,514293,0,2190584004,Tue Jun 16 03:08:48 PDT 2009,NO_QUERY,Vicki_Gee,i miss nikki nu nu already shes always there ...
1,1,142282,0,1881451988,Fri May 22 04:42:15 PDT 2009,NO_QUERY,PatCashin,So I had a dream last night. I remember a sig...
2,2,403727,0,2058252964,Sat Jun 06 14:34:17 PDT 2009,NO_QUERY,deelectable,@girlyghost ohh poor sickly you (((hugs)) ho...
3,3,649503,0,2237307600,Fri Jun 19 05:34:22 PDT 2009,NO_QUERY,justinekepa,it is raining again
4,4,610789,0,2224301193,Thu Jun 18 09:20:06 PDT 2009,NO_QUERY,cmatt007,@MissKeriBaby wish I was in LA right now
5,5,67315,0,1692257495,Sun May 03 19:36:35 PDT 2009,NO_QUERY,Dogbook,Nala Olowalu still has a full tummy from bread...
6,6,833521,4,1557782222,Sun Apr 19 05:58:52 PDT 2009,NO_QUERY,MyAppleStuff,@macintom site doesn't seem to want to load up...
7,7,256032,0,1984628467,Sun May 31 15:29:11 PDT 2009,NO_QUERY,AlexCarrier,time for some sleep- hav to actually do some w...
8,8,657012,0,2240793149,Fri Jun 19 10:10:09 PDT 2009,NO_QUERY,simonhowes,@supercoolkp In Oxford that month.
9,9,980587,4,1834037031,Mon May 18 01:23:05 PDT 2009,NO_QUERY,Sofuce,"..time for a cup of tea and fruit bagels, i'm ..."


In [33]:
df.text

0        i miss nikki nu nu already  shes always there ...
1        So I had a dream last night. I  remember a sig...
2        @girlyghost ohh poor sickly you   (((hugs)) ho...
3                                     it is raining again 
4                @MissKeriBaby wish I was in LA right now 
                               ...                        
19995    I get the worst writer's cramp. These thank yo...
19996          @ladyinreddress the sun is all gone now... 
19997    FINALLY a break till Wed, no work OR school, b...
19998                              I want a Blackberry... 
19999    in montrea doing some much needed ocean swimmi...
Name: text, Length: 20000, dtype: object

In [34]:
ALL_LETTERS=string.ascii_letters+" .,;'"
N_LETTERS=len(ALL_LETTERS)

In [35]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD',s)
                   if unicodedata.category(c)!='Mn'
                   and c in ALL_LETTERS
                   )

In [36]:
#converting all letters to acii ]
for i in range(len(df)):
    text=df.text[i].split()
    string=[]
    for words in text:
        words=unicode_to_ascii(words)
        words=words.lower()
        string.append(words)

    df.text[i]=' '.join(string)

    



/tmp/ipykernel_16995/3986324385.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.text[i]=' '.join(string)


In [37]:
df.head(10)

,Unnamed: 0,index,target,ids,date,flag,user,text
0,0,514293,0,2190584004,Tue Jun 16 03:08:48 PDT 2009,NO_QUERY,Vicki_Gee,i miss nikki nu nu already shes always there w...
1,1,142282,0,1881451988,Fri May 22 04:42:15 PDT 2009,NO_QUERY,PatCashin,so i had a dream last night. i remember a sign...
2,2,403727,0,2058252964,Sat Jun 06 14:34:17 PDT 2009,NO_QUERY,deelectable,girlyghost ohh poor sickly you hugs hope you f...
3,3,649503,0,2237307600,Fri Jun 19 05:34:22 PDT 2009,NO_QUERY,justinekepa,it is raining again
4,4,610789,0,2224301193,Thu Jun 18 09:20:06 PDT 2009,NO_QUERY,cmatt007,misskeribaby wish i was in la right now
5,5,67315,0,1692257495,Sun May 03 19:36:35 PDT 2009,NO_QUERY,Dogbook,nala olowalu still has a full tummy from bread...
6,6,833521,4,1557782222,Sun Apr 19 05:58:52 PDT 2009,NO_QUERY,MyAppleStuff,"macintom site doesn't seem to want to load up,..."
7,7,256032,0,1984628467,Sun May 31 15:29:11 PDT 2009,NO_QUERY,AlexCarrier,time for some sleep hav to actually do some wo...
8,8,657012,0,2240793149,Fri Jun 19 10:10:09 PDT 2009,NO_QUERY,simonhowes,supercoolkp in oxford that month.
9,9,980587,4,1834037031,Mon May 18 01:23:05 PDT 2009,NO_QUERY,Sofuce,"..time for a cup of tea and fruit bagels, i'm ..."


In [38]:
for i in range(len(df)):
    df.text[i]=re.sub(r"\S*https?:\S*"," ",df.text[i])
    df.text[i]=re.sub(r"[^a-zA-Z0-9]"," ",df.text[i])



/tmp/ipykernel_16995/1551427966.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.text[i]=re.sub(r"\S*https?:\S*"," ",df.text[i])
/tmp/ipykernel_16995/1551427966.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.text[i]=re.sub(r"[^a-zA-Z0-9]"," ",df.text[i])


In [39]:
df.head(10)

,Unnamed: 0,index,target,ids,date,flag,user,text
0,0,514293,0,2190584004,Tue Jun 16 03:08:48 PDT 2009,NO_QUERY,Vicki_Gee,i miss nikki nu nu already shes always there w...
1,1,142282,0,1881451988,Fri May 22 04:42:15 PDT 2009,NO_QUERY,PatCashin,so i had a dream last night i remember a sign...
2,2,403727,0,2058252964,Sat Jun 06 14:34:17 PDT 2009,NO_QUERY,deelectable,girlyghost ohh poor sickly you hugs hope you f...
3,3,649503,0,2237307600,Fri Jun 19 05:34:22 PDT 2009,NO_QUERY,justinekepa,it is raining again
4,4,610789,0,2224301193,Thu Jun 18 09:20:06 PDT 2009,NO_QUERY,cmatt007,misskeribaby wish i was in la right now
5,5,67315,0,1692257495,Sun May 03 19:36:35 PDT 2009,NO_QUERY,Dogbook,nala olowalu still has a full tummy from bread...
6,6,833521,4,1557782222,Sun Apr 19 05:58:52 PDT 2009,NO_QUERY,MyAppleStuff,macintom site doesn t seem to want to load up ...
7,7,256032,0,1984628467,Sun May 31 15:29:11 PDT 2009,NO_QUERY,AlexCarrier,time for some sleep hav to actually do some wo...
8,8,657012,0,2240793149,Fri Jun 19 10:10:09 PDT 2009,NO_QUERY,simonhowes,supercoolkp in oxford that month
9,9,980587,4,1834037031,Mon May 18 01:23:05 PDT 2009,NO_QUERY,Sofuce,time for a cup of tea and fruit bagels i m ...


In [55]:
tokenizer=BertTokenizerFast.from_pretrained("setu4993/LaBSE")
model=BertModel.from_pretrained("setu4993/LaBSE")

In [56]:
device=torch.device("cuda")

In [57]:
#class for dataloader 
from torch.utils.data.dataloader import default_collate
class NaiveDataLoader:
    def __init__(self, dataset, batch_size=64, collate_fn=default_collate):
        self.dataset = dataset
        self.batch_size = batch_size
        self.collate_fn = collate_fn
        self.index = 0
    def __iter__(self):
        self.index = 0
        return self

    def __next__(self):
        if self.index >= len(self.dataset):
            # stop iteration once index is out of bounds
            raise StopIteration
        batch_size = min(len(self.dataset) - self.index, self.batch_size)
        return self.collate_fn([self.get() for _ in range(batch_size)])

    def get(self):
        item = self.dataset[self.index]
        self.index += 1
        return item

In [58]:
corpus=[df.text[i] for i in range(len(df))]

In [59]:
corpus_loader=NaiveDataLoader(corpus,batch_size=500)

In [62]:
output_embedings=torch.tensor([]).to(device)

In [63]:
for text in corpus_loader:
  input=tokenizer(text, return_tensors="pt", padding=True)
  input_ids=input.input_ids.clone().detach()
  token_type_ids=input.token_type_ids.clone().detach()
  attention_mask=input.attention_mask.clone().detach()
  input_ids=input_ids.to(device)
  token_type_ids=token_type_ids.to(device)
  attention_mask=attention_mask.to(device)
  model=model.to(device)
  with torch.no_grad():
    output=model(input_ids,token_type_ids,attention_mask)
    output_embedings=torch.cat((output_embedings,output.pooler_output))
    torch.cuda.empty_cache()
  with torch.no_grad():
    torch.cuda.empty_cache()

In [64]:
output_embedings.shape

torch.Size([20000, 768])

In [65]:
torch.save(output_embedings,"embedings.pt")

In [72]:
labels=np.array(df.target)

In [73]:
labels_tensor=torch.from_numpy(labels)

In [74]:
torch.save(labels_tensor,";labels.pt")

In [4]:
X=torch.load('embedings.pt')

In [7]:
torch.max(X[0])

tensor(0.9651, device='cuda:0')